# Processing data for modelling

## - Stock price

In [1]:
import pandas as pd
from tqdm import tqdm

from functions import print_shape


In [2]:
# test with feature_engineering.py
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
print_shape(prices)
prices.head(2)

 Shape:
 ----------------------------------------
 Observations:   2.33M
 Features:       12
 Feature Date:    datetime64[ns]


,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324


In [3]:

from feature_engineering import fill_and_drop_na_values

fill_prices = fill_and_drop_na_values(prices)
print_shape(fill_prices)
fill_prices.head(2)


 87%|████████▋ | 1745/2000 [04:15<02:49,  1.51it/s]

In [ ]:
from feature_engineering import adjust_price

ad_price = adjust_price(fill_prices)
print_shape(ad_price)
ad_price.head(2)

In [ ]:
ad_price.to_csv('data/curr_ad_price.csv') # TODO index column as index

In [ ]:
import pandas as pd
from tqdm import tqdm

from functions import print_shape
ad_price = pd.read_csv('data/curr_ad_price.csv', parse_dates=['Date'])
print_shape(ad_price)

In [ ]:
#from importlib import reload
#import feature_engineering as ft

from feature_engineering import price_new_features

#ft = reload(ft)
ad_price_feat = price_new_features(ad_price)
print_shape(ad_price_feat)

In [ ]:
type(ad_price_feat)

In [ ]:
ad_price_feat.to_csv('data/curr_ad_price_feat.csv')

In [ ]:
from feature_engineering import encode_flag

ad_price_feat['SupervisionFlag'] = encode_flag(ad_price_feat)
print_shape(ad_price)

In [ ]:
ad_price_feat.columns

## - Financials

In [ ]:
#from feature_engineering import fill_finances

financial = financial = pd.read_csv('data/train_files/financials.csv',parse_dates=[2])
financial.head(2)

In [ ]:
from feature_engineering import fill_finances_knn



filled_finances = fill_finances_knn(financial, prices)
filled_finances.head()
filled_finances.shape

In [ ]:
# ffill and drop 


def new_features_financial(filled_finances):
    sec_codes = filled_finances.SecuritiesCode.unique()

    filled_financial_feat = pd.DataFrame(columns=filled_finances.columns)

    for i in tqdm(sec_codes):
        # select a security code
        aktie = filled_finances.query('SecuritiesCode == @i')
        aktie.sort_values('Date')
        # create new features:
        aktie['margin'] = aktie['Profit'] / aktie['NetSales'] * 100
        aktie['profit_ttm'] = aktie['Profit'].shift(3) + aktie['Profit'].shift(2) + aktie['Profit'].shift(1) + aktie['Profit']
        aktie['rev_ttm'] = aktie['NetSales'].shift(3) + aktie['NetSales'].shift(2) + aktie['NetSales'].shift(1) + aktie['NetSales']
        aktie['win_quarter_growth'] = (aktie['Profit'] - aktie['Profit'].shift(1)) / aktie['Profit'].shift(1) * 100
        aktie['rev_quarter_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(1)) / aktie['NetSales'].shift(1) * 100
        aktie['win_yoy_growth'] = (aktie['Profit'] - aktie['Profit'].shift(4)) / aktie['Profit'].shift(4) * 100
        aktie['rev_yoy_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(4)) / aktie['NetSales'].shift(4) * 100
        aktie['win_ttm_growth'] = (aktie['profit_ttm'] - aktie['profit_ttm'].shift(1)) / aktie['profit_ttm'].shift(1) * 100
        aktie['rev_ttm_growth'] = (aktie['rev_ttm'] - aktie['rev_ttm'].shift(1)) / aktie['rev_ttm'].shift(1) * 100

        # fill
        aktie = aktie.ffill()
        aktie = aktie.dropna(axis=0)

        filled_financial_feat  = pd.concat([filled_financial_feat , aktie])

        filled_financial_feat['Date'] = pd.to_datetime(filled_financial_feat['Date']) 
    
    return filled_financial_feat

In [ ]:
filled_financial_features = new_features_financial(filled_finances)

In [ ]:
"""from functions import plot_stock
code = 1301

plot_stock(filled_financial_features, code,'win_ttm_growth' )"""

In [ ]:
filled_financial_features.head(2)


In [ ]:
filled_financial_features.SecuritiesCode = filled_financial_features.SecuritiesCode.astype(int)

In [ ]:
filled_financial_features.head(2)

In [ ]:
ad_price.head(2)

## - financials + price

In [ ]:
# create key on financial : RowId
filled_financial_features['RowId'] = filled_financial_features.Date.dt.strftime('%Y%m%d').astype(str) + '_' + filled_financial_features.SecuritiesCode.astype(str)

In [ ]:
#filled_financial_features.sort_values('Date').head(20)

In [ ]:
filled_financial_features.RowID.info()

In [ ]:
# concat financials and price
price_financial = pd.merge(ad_price, filled_financial_features, how='left', on='RowId', suffixes=[None, 'f_'])
#price_financial = pd.concat([ad_price, filled_financial_features], keys='RowID')

In [ ]:
price_financial.shape

In [ ]:
price_financial.head()

In [ ]:
price_financial.query('RowID == "20180209_1301"')


In [ ]:
price_financial.shape

In [ ]:
from functions import missingValues

missingValues(price_financial)

In [ ]:
# ffill and drop 

def fill_drop_pro_code(df):
    sec_codes = df.SecuritiesCode.unique()

    df_fill_na = pd.DataFrame(columns=price_financial.columns)

    for i in tqdm(sec_codes):

        current = df.query('SecuritiesCode == @i')
        current.sort_values('Date')
        current = current.ffill()
        current = current.dropna(axis=0)

        df_fill_na  = pd.concat([df_fill_na , current])

    df_fill_na['Date'] = pd.to_datetime(df_fill_na['Date'])
    
    return df_fill_na

In [ ]:
price_financial_fill = fill_drop_pro_code(price_financial)

In [ ]:
# new features
#price_financial_fill.drop_duplicates(inplace=True)
price_financial_fill.shape

In [ ]:
# save dataframe to csv
price_financial_fill.to_csv('data/current.csv')